#Data ingestion into neo4j and data analysis via spark

####Import required libraries

In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains

In [0]:
spark = SparkSession.builder.appName('PySpark Read CSV').getOrCreate()

####Import home loan data from csv and load into dataframes 
####Take key columns required for building relationships and load into data frames

In [0]:
import pandas as pd
df_full = spark.read.options(header='True', delimiter=',') \
              .csv("/FileStore/tables/home_loan_50_rows.csv")

df_home_loan=df_full.select("home_loan_id","tenure","rate","dpd","ltv","soi_cohort_employee","soi_cohort_online_heavy","soi_cohort_workplace","soi_cohort_millitary_banking","soi_cohort_student","orig_dt","maturity_dt","sch_amt","amount_owed","loan_status")


df_relationship = df_full.select("customer_id","home_loan_id","orig_amt")
df_home_loan.show()

+------------+------+-----------+------+-----------+-------------------+-----------------------+--------------------+----------------------------+------------------+----------+-----------+-------+-----------+-----------+
|home_loan_id|tenure|       rate|   dpd|        ltv|soi_cohort_employee|soi_cohort_online_heavy|soi_cohort_workplace|soi_cohort_millitary_banking|soi_cohort_student|   orig_dt|maturity_dt|sch_amt|amount_owed|loan_status|
+------------+------+-----------+------+-----------+-------------------+-----------------------+--------------------+----------------------------+------------------+----------+-----------+-------+-----------+-----------+
|    HL868849|    25|0.090291502| DPD_0|0.201491365|                 NA|                     88|                  68|                          87|                40|29-07-2008| 28-06-2033|1152.61|          0|    InForce|
|    HL537584|    25|0.175185036| DPD_0|1.014000854|                 NA|                     47|                  NA

In [0]:
df_relationship.show()

+-----------+------------+--------+
|customer_id|home_loan_id|orig_amt|
+-----------+------------+--------+
| CUST235925|    HL868849|  341897|
| CUST290194|    HL537584|  406327|
| CUST288657|    HL464229|  327103|
| CUST301895|    HL527310|  267019|
| CUST747474|    HL159536|   89522|
| CUST411524|    HL898643|  339070|
| CUST643840|    HL862570|  412542|
| CUST472418|    HL208697|  484532|
| CUST164089|    HL681200|   84221|
| CUST850621|    HL805092|  493150|
| CUST301488|    HL199671|  293295|
| CUST254353|    HL428067|  334204|
| CUST854589|    HL122253|  142169|
| CUST660250|    HL661959|  229148|
| CUST109556|    HL424297|  494377|
| CUST219645|    HL441567|  350485|
| CUST346042|    HL912850|  121973|
| CUST537213|    HL139128|  475770|
| CUST556443|    HL968443|  174239|
| CUST594764|    HL426857|  186185|
+-----------+------------+--------+
only showing top 20 rows



####Import customer data from csv and load into dataframe

In [0]:
df_customer = spark.read.options(header='True', delimiter=',') \
              .csv("/FileStore/tables/customer_data_50_rows.csv")
df_customer.show()

+-----------+---+--------+------+----+-----------+-------------------+---------------+-------------------+---------------------------+-------------------+----------+------+----------+--------------------+--------+----------+--------------------+
|customer_id|age|location|gender|risk|fico_scores|       wallet_share|income_estimate|          dti_ratio|average_time_spent(minutes)|         last_login|     phone|   zip|       ssn|    action_performed|   fname|     lname|               email|
+-----------+---+--------+------+----+-----------+-------------------+---------------+-------------------+---------------------------+-------------------+----------+------+----------+--------------------+--------+----------+--------------------+
| CUST301488| 36|      MN|     M|   M|        428| 0.2144165972873382|         112882| 0.8747584324682289|                         81|2017-07-08 23:02:23|7793364908|419154|8887475444|        Pay with Tap|   James|    Rogers|james.rogers@olso...|
| CUST235925| 36

#####Write Home loan dataframe to neo4j

In [0]:
df_home_loan.write\
  .format("org.neo4j.spark.DataSource")\
  .mode("Overwrite")\
  .option("authentication.type", "basic")\
   .option("url", "bolt://3.91.209.92:7687")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "housings-eliminators-washing")\
  .option("labels", "Home_Loan")\
  .option("node.keys","home_loan_id")\
  .save()

####Write customer dataframe to neo4j

In [0]:
df_customer.write\
  .format("org.neo4j.spark.DataSource")\
  .mode("Overwrite")\
  .option("authentication.type", "basic")\
  .option("url", "bolt://18.214.36.252:7687")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "editors-seamanship-wounds")\
  .option("labels", "Customer")\
  .option("node.keys","customer_id")\
  .save()

#####Create relationship between customer and home loan in neo4j

In [0]:

df_relationship.write\
  .format("org.neo4j.spark.DataSource")\
  .mode("Overwrite")\
  .option("authentication.type", "basic")\
  .option("url", "bolt://18.214.36.252:7687")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "editors-seamanship-wounds")\
  .option("relationship", "HAS_APPLIED")\
  .option("relationship.save.strategy", "keys")\
  .option("relationship.source.labels", "Customer")\
  .option("relationship.source.node.keys", "customer_id")\
  .option("relationship.source.save.mode", "overwrite")\
  .option("relationship.target.labels", "Home_Loan")\
  .option("relationship.target.node.keys", "home_loan_id")\
  .option("relationship.properties","orig_amt")\
  .option("relationship.target.save.mode", "overwrite")\
  .save()

####Write the same data for homeloan again to verify if its overwriting old data

In [0]:

df_home_loan.write\
  .format("org.neo4j.spark.DataSource")\
  .mode("Overwrite")\
  .option("authentication.type", "basic")\
  .option("url", "bolt://18.214.36.252:7687")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "editors-seamanship-wounds")\
  .option("labels", "Home_Loan")\
  .option("node.keys","home_loan_id")\
  .save()

####Write the same data for customer again to verify if its overwriting old data

In [0]:

df_customer.write\
  .format("org.neo4j.spark.DataSource")\
  .mode("Overwrite")\
  .option("authentication.type", "basic")\
  .option("url", "bolt://18.214.36.252:7687")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "editors-seamanship-wounds")\
  .option("labels", "Customer")\
  .option("node.keys","customer_id")\
  .save()

#### Write the same data for relationship again to verify if its overwriting old data

In [0]:
df_relationship.write\
  .format("org.neo4j.spark.DataSource")\
  .mode("Overwrite")\
  .option("authentication.type", "basic")\
  .option("url", "bolt://18.214.36.252:7687")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "editors-seamanship-wounds")\
  .option("relationship", "HAS_APPLIED")\
  .option("relationship.save.strategy", "keys")\
  .option("relationship.source.labels", "Customer")\
  .option("relationship.source.node.keys", "customer_id")\
  .option("relationship.source.save.mode", "overwrite")\
  .option("relationship.target.labels", "Home_Loan")\
  .option("relationship.target.node.keys", "home_loan_id")\
  .option("relationship.properties","orig_amt")\
  .option("relationship.target.save.mode", "overwrite")\
  .save()

##Data Manipulation Queries on Neo4j data

####Add one new record in each file and Try to load the same files – check it will perform upsert?
####Already we have 50 records in the customer and home loan lets add 2 more records in each of them
####Data preparation for getting updated data for home loan and relationship

In [0]:
import pandas as pd
df_full_updated = spark.read.options(header='True', delimiter=',') \
              .csv("/FileStore/tables/home_loan_52_rows.csv")

df_home_loan_52=df_full_updated.select("home_loan_id","tenure","rate","dpd","ltv","soi_cohort_employee","soi_cohort_online_heavy","soi_cohort_workplace","soi_cohort_millitary_banking","soi_cohort_student","orig_dt","maturity_dt","sch_amt","amount_owed","loan_status")
display(df_home_loan_52.count())

df_relationship_52 = df_full_updated.select("customer_id","home_loan_id","orig_amt")

52

####Data preparation for getting updated data for customer

In [0]:

df_customer_52 = spark.read.options(header='True', delimiter=',') \
              .csv("/FileStore/tables/customer_data_52_rows.csv")
display(df_customer_52.count())

52

####Write the new data to check whether it will upsert the data
####Write Homeloan
####it inserted 2 records into neo4j plus 50 existing records,so ending up as 52 nodes.

In [0]:

df_home_loan_52.write\
  .format("org.neo4j.spark.DataSource")\
  .mode("Overwrite")\
  .option("authentication.type", "basic")\
  .option("url", "bolt://18.214.36.252:7687")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "editors-seamanship-wounds")\
  .option("labels", "Home_Loan")\
  .option("node.keys","home_loan_id")\
  .save()


####Write the new data to check whether it will upsert the data
####Write customer data

In [0]:
df_customer_52.write\
  .format("org.neo4j.spark.DataSource")\
  .mode("Overwrite")\
  .option("authentication.type", "basic")\
  .option("url", "bolt://18.214.36.252:7687")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "editors-seamanship-wounds")\
  .option("labels", "Customer")\
  .option("node.keys","customer_id")\
  .save()

####Write the new data to check whether it will upsert the data
####Relationship data will add 2 more relationship

In [0]:
df_relationship_52.write\
  .format("org.neo4j.spark.DataSource")\
  .mode("Overwrite")\
  .option("authentication.type", "basic")\
  .option("url", "bolt://18.214.36.252:7687")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "editors-seamanship-wounds")\
  .option("relationship", "HAS_APPLIED")\
  .option("relationship.save.strategy", "keys")\
  .option("relationship.source.labels", "Customer")\
  .option("relationship.source.node.keys", "customer_id")\
  .option("relationship.source.save.mode", "overwrite")\
  .option("relationship.target.labels", "Home_Loan")\
  .option("relationship.target.node.keys", "home_loan_id")\
  .option("relationship.properties","orig_amt")\
  .option("relationship.target.save.mode", "overwrite")\
  .save()

##Update any one record based on the customer id

####Data preparation to update the data.

In [0]:
import pandas as pd
data = [["CUST301488", "Justin","Park"], ["CUST235925", "Don","Benjamin"], ["CUST768307", "Evan","Lewis"]]
pdf = pd.DataFrame(data, columns=["id", "firstname","lastname"])
data_to_be_updated = spark.createDataFrame(pdf)
display(data_to_be_updated)

id,firstname,lastname
CUST301488,Justin,Park
CUST235925,Don,Benjamin
CUST768307,Evan,Lewis


####Read data from neo4j before updating from spark

In [0]:

spark.read.format("org.neo4j.spark.DataSource")\
       .option("authentication.type", "basic")\
      .option("url", "bolt://18.214.36.252:7687")\
      .option("authentication.basic.username", "neo4j")\
      .option("authentication.basic.password", "editors-seamanship-wounds")\
       .option("query","MATCH (c:Customer) WHERE c.customer_id in ['CUST301488','CUST235925','CUST768307']RETURN\
               c.customer_id,c.fname,c.lname")\
        .load()\
        .show()
        

+-------------+--------+-------+
|c.customer_id| c.fname|c.lname|
+-------------+--------+-------+
|   CUST301488|   James| Rogers|
|   CUST235925|Margaret|Vasquez|
|   CUST768307|   Derek| Rivera|
+-------------+--------+-------+



####Update data for the given customer ids

In [0]:

data_to_be_updated.write\
  .format("org.neo4j.spark.DataSource")\
  .mode("Overwrite")\
  .option("authentication.type", "basic")\
  .option("url", "bolt://18.214.36.252:7687")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "editors-seamanship-wounds")\
  .option("query", "MATCH(c:Customer{customer_id:event.id}) SET c.fname=event.firstname, c.lname=event.lastname")\
   .save()
  

####After update, read the data for the same customer ids

In [0]:
spark.read.format("org.neo4j.spark.DataSource")\
       .option("authentication.type", "basic")\
      .option("url", "bolt://18.214.36.252:7687")\
      .option("authentication.basic.username", "neo4j")\
      .option("authentication.basic.password", "editors-seamanship-wounds")\
       .option("query","MATCH (c:Customer) WHERE c.customer_id in ['CUST301488','CUST235925','CUST768307']RETURN\
               c.customer_id,c.fname,c.lname")\
        .load()\
        .show()

+-------------+-------+--------+
|c.customer_id|c.fname| c.lname|
+-------------+-------+--------+
|   CUST301488| Justin|    Park|
|   CUST235925|    Don|Benjamin|
|   CUST768307|   Evan|   Lewis|
+-------------+-------+--------+



##Delete a record based on customer id

####Delete the customer ids CUST301488,CUST235925 and CUST768307

In [0]:
data_to_be_updated.write\
  .format("org.neo4j.spark.DataSource")\
  .mode("Overwrite")\
  .option("authentication.type", "basic")\
  .option("url", "bolt://18.214.36.252:7687")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "editors-seamanship-wounds")\
  .option("query", "MATCH(c:Customer{customer_id:event.id}) DETACH DELETE c")\
  .save()
  

####Verify if data is deleted

In [0]:


spark.read.format("org.neo4j.spark.DataSource")\
       .option("authentication.type", "basic")\
      .option("url", "bolt://18.214.36.252:7687")\
      .option("authentication.basic.username", "neo4j")\
      .option("authentication.basic.password", "editors-seamanship-wounds")\
       .option("query","MATCH (c:Customer) WHERE c.customer_id in ['CUST301488','CUST235925','CUST768307']RETURN\
               c.customer_id,c.fname,c.lname")\
        .load()\
        .show()

+-------------+-------+-------+
|c.customer_id|c.fname|c.lname|
+-------------+-------+-------+
+-------------+-------+-------+



##Read the data from node4j to dataframe and display it

####Read the data from node4j to dataframe and display it
####Read customer data

In [0]:

df_read_customer = spark.read.format("org.neo4j.spark.DataSource")\
 .option("authentication.type", "basic")\
 .option("url", "bolt://18.214.36.252:7687")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "editors-seamanship-wounds")\
 .option("labels", "Customer")\
 .load()
display(df_read_customer)

,,zip,wallet_share,ssn,risk,phone,location,lname,last_login,income_estimate,gender,fname,fico_scores,email,dti_ratio,customer_id,average_time_spent(minutes),age,action_performed
53,List(Customer),236362,0.941966643,8475385450,L,650494074,OK,Campbell,29-01-2013 18:17,112343,F,Sarah,702,sarah.campbell@reese.com,0.429730273,CUST854589,39,18,Help Center
54,List(Customer),670405,0.411026145,267725099,F,9728883573,CT,Lara,08-04-2019 08:18,113523,F,Amanda,649,amanda.lara@downs-young.com,0.626941097,CUST288657,15,22,Pay with Tap
55,List(Customer),864604,0.992088335,6809263436,L,5236363153,PA,Jones,02-04-2011 04:59,97164,F,Jennifer,315,jennifer.jones@taylor-anderson.com,0.790688528,CUST301895,20,35,Bill Payments
56,List(Customer),79262,0.873145174,1918835875,L,8995232999,WA,Gonzalez,11-04-2018 18:35,108038,F,Beverly,724,beverly.gonzalez@thomas.com,0.092366208,CUST747228,62,28,Transaction Statements
57,List(Customer),135546,0.771350124,2295650717,L,2333931362,MS,Gibbs,15-08-2012 02:27,94656,F,Lisa,645,lisa.gibbs@townsend.com,0.939647569,CUST626632,36,45,Zelle QR Code
58,List(Customer),764390,0.898435692,3581714686,F,5916176340,ME,Duarte,20-12-2011 07:34,112199,F,Kiara,470,kiara.duarte@bradford.org,0.670086718,CUST648137,15,54,Help Center
59,List(Customer),754909,0.102420743,7099737570,F,955885263,CO,Lawson,26-07-2019 23:22,111066,M,John,430,john.lawson@allen.com,0.977855526,CUST499509,24,36,Help Center
60,List(Customer),457203,0.130273946,6068181069,M,7300372011,OH,Frey,11-04-2010 07:09,96535,M,Donald,775,donald.frey@stevens.net,0.82738475,CUST164089,48,32,Easy Lock
61,List(Customer),157120,0.899833103,7672994991,M,7327517357,MS,Hall,28-07-2013 03:19,96709,F,Susan,472,susan.hall@wolfe.biz,0.788840699,CUST215330,85,18,Help Center
62,List(Customer),778999,0.25493794,1037603771,F,525476139,AL,Perry,10-01-2014 09:55,92632,F,Jocelyn,658,jocelyn.perry@walsh-huber.com,0.895011178,CUST594764,37,20,Transfer Funds


####Read the data from node4j to dataframe and display it
####Read homeloan data

In [0]:

df_read_homeloan = spark.read.format("org.neo4j.spark.DataSource")\
 .option("authentication.type", "basic")\
 .option("url", "bolt://18.214.36.252:7687")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "editors-seamanship-wounds")\
 .option("labels", "Home_Loan")\
 .load()
display(df_read_homeloan)

,,tenure,soi_cohort_workplace,soi_cohort_student,soi_cohort_online_heavy,soi_cohort_millitary_banking,soi_cohort_employee,sch_amt,rate,orig_dt,maturity_dt,ltv,loan_status,home_loan_id,dpd,amount_owed
0,List(Home_Loan),25,68,40,88,87,NA,1152.61,0.090291502,29-07-2008,28-06-2033,0.201491365,InForce,HL868849,DPD_0,0
1,List(Home_Loan),25,NA,NA,47,NA,NA,1384.4,0.175185036,27-03-2015,24-02-2040,1.014000854,InForce,HL537584,DPD_0,0
2,List(Home_Loan),12,99,52,NA,57,62,2296.66,0.182285179,25-05-2016,05-10-2028,1.485103614,InForce,HL464229,DPD_0,0
3,List(Home_Loan),25,78,NA,NA,NA,NA,902.57,0.111554415,06-08-2015,07-05-2040,0.992136285,InForce,HL527310,DPD_0,0
4,List(Home_Loan),10,77,NA,NA,55,NA,753.49,0.198102549,12-11-2017,31-10-2027,0.810476429,InForce,HL159536,DPD_0,0
5,List(Home_Loan),25,NA,90,NA,98,NA,1156.48,0.183737854,16-09-2010,16-08-2035,0.661556655,InForce,HL898643,DPD_0,0
6,List(Home_Loan),12,77,69,49,71,50,2875.49,0.061235838,10-05-2011,25-04-2023,1.40144415,InForce,HL862570,DPD_0,0
7,List(Home_Loan),25,51,81,55,NA,83,1655.52,0.197888007,12-07-2010,06-11-2035,0.165780049,InForce,HL208697,DPD_0,0
8,List(Home_Loan),20,94,NA,81,96,63,355.76,0.136640174,07-11-2011,13-10-2031,0.166789872,InForce,HL681200,DPD_0,0
9,List(Home_Loan),15,57,NA,NA,NA,NA,2768.31,0.137892239,07-11-2009,19-10-2024,0.355470777,InForce,HL805092,DPD_0,0


####Read via relationships and load into dataframe

In [0]:

df_read_relationship=spark.read.format("org.neo4j.spark.DataSource")\
       .option("authentication.type", "basic")\
       .option("url", "bolt://18.214.36.252:7687")\
      .option("authentication.basic.username", "neo4j")\
      .option("authentication.basic.password", "editors-seamanship-wounds")\
       .option("relationship","HAS_APPLIED")\
        .option("relationship.source.labels","Customer")\
        .option("relationship.target.labels","Home_Loan")\
        .load()



####Display dataframe holding relationship

In [0]:
display(df_read_relationship)

,,,,source.zip,source.wallet_share,source.ssn,source.risk,source.phone,source.location,source.lname,source.last_login,source.income_estimate,source.gender,source.fname,source.fico_scores,source.email,source.dti_ratio,source.customer_id,source.average_time_spent(minutes),source.age,source.action_performed,,,target.tenure,target.soi_cohort_workplace,target.soi_cohort_student,target.soi_cohort_online_heavy,target.soi_cohort_millitary_banking,target.soi_cohort_employee,target.sch_amt,target.rate,target.orig_dt,target.maturity_dt,target.ltv,target.loan_status,target.home_loan_id,target.dpd,target.amount_owed,rel.orig_amt
12,HAS_APPLIED,53,List(Customer),236362,0.941966643,8475385450,L,650494074,OK,Campbell,29-01-2013 18:17,112343,F,Sarah,702,sarah.campbell@reese.com,0.429730273,CUST854589,39,18,Help Center,12,List(Home_Loan),12,60,74,NA,NA,NA,991.33,0.067780103,02-03-2009,15-02-2021,1.36460131,Matured,HL122253,DPD_0,0,142169
2,HAS_APPLIED,54,List(Customer),670405,0.411026145,267725099,F,9728883573,CT,Lara,08-04-2019 08:18,113523,F,Amanda,649,amanda.lara@downs-young.com,0.626941097,CUST288657,15,22,Pay with Tap,2,List(Home_Loan),12,99,52,NA,57,62,2296.66,0.182285179,25-05-2016,05-10-2028,1.485103614,InForce,HL464229,DPD_0,0,327103
3,HAS_APPLIED,55,List(Customer),864604,0.992088335,6809263436,L,5236363153,PA,Jones,02-04-2011 04:59,97164,F,Jennifer,315,jennifer.jones@taylor-anderson.com,0.790688528,CUST301895,20,35,Bill Payments,3,List(Home_Loan),25,78,NA,NA,NA,NA,902.57,0.111554415,06-08-2015,07-05-2040,0.992136285,InForce,HL527310,DPD_0,0,267019
26,HAS_APPLIED,56,List(Customer),79262,0.873145174,1918835875,L,8995232999,WA,Gonzalez,11-04-2018 18:35,108038,F,Beverly,724,beverly.gonzalez@thomas.com,0.092366208,CUST747228,62,28,Transaction Statements,26,List(Home_Loan),15,NA,NA,92,70,NA,2155.21,0.080322179,21-07-2019,07-02-2034,1.49441951,InForce,HL872988,DPD_0,0,385598
44,HAS_APPLIED,57,List(Customer),135546,0.771350124,2295650717,L,2333931362,MS,Gibbs,15-08-2012 02:27,94656,F,Lisa,645,lisa.gibbs@townsend.com,0.939647569,CUST626632,36,45,Zelle QR Code,44,List(Home_Loan),10,NA,NA,NA,89,NA,4124.41,0.158518348,10-11-2018,28-10-2028,0.074213552,InForce,HL195807,DPD_0,0,490995
20,HAS_APPLIED,58,List(Customer),764390,0.898435692,3581714686,F,5916176340,ME,Duarte,20-12-2011 07:34,112199,F,Kiara,470,kiara.duarte@bradford.org,0.670086718,CUST648137,15,54,Help Center,20,List(Home_Loan),25,NA,99,NA,40,NA,1180.2,0.090155594,21-04-2018,21-03-2043,0.844968676,InForce,HL431682,DPD_0,0,350086
27,HAS_APPLIED,59,List(Customer),754909,0.102420743,7099737570,F,955885263,CO,Lawson,26-07-2019 23:22,111066,M,John,430,john.lawson@allen.com,0.977855526,CUST499509,24,36,Help Center,27,List(Home_Loan),12,64,54,64,69,79,517.06,0.193395753,12-02-2012,28-01-2024,0.279983898,InForce,HL560265,DPD_90,1551.18,73593
8,HAS_APPLIED,60,List(Customer),457203,0.130273946,6068181069,M,7300372011,OH,Frey,11-04-2010 07:09,96535,M,Donald,775,donald.frey@stevens.net,0.82738475,CUST164089,48,32,Easy Lock,8,List(Home_Loan),20,94,NA,81,96,63,355.76,0.136640174,07-11-2011,13-10-2031,0.166789872,InForce,HL681200,DPD_0,0,84221
22,HAS_APPLIED,61,List(Customer),157120,0.899833103,7672994991,M,7327517357,MS,Hall,28-07-2013 03:19,96709,F,Susan,472,susan.hall@wolfe.biz,0.788840699,CUST215330,85,18,Help Center,22,List(Home_Loan),15,64,NA,NA,54,NA,2794.2,0.134611642,31-08-2016,13-08-2031,0.899568954,InForce,HL458584,DPD_0,0,497884
19,HAS_APPLIED,62,List(Customer),778999,0.25493794,1037603771,F,525476139,AL,Perry,10-01-2014 09:55,92632,F,Jocelyn,658,jocelyn.perry@walsh-huber.com,0.895011178,CUST594764,37,20,Transfer Funds,19,List(Home_Loan),12,NA,NA,NA,NA,NA,1299.27,0.080715477,06-09-2009,22-08-2021,0.448225857,Overdue,HL426857,DPD_90,3897.81,186185


####Read via cypher queries and load into dataframe

In [0]:

df_read_home_loan_data=spark.read.format("org.neo4j.spark.DataSource")\
       .option("authentication.type", "basic")\
       .option("url", "bolt://18.214.36.252:7687")\
      .option("authentication.basic.username", "neo4j")\
      .option("authentication.basic.password", "editors-seamanship-wounds")\
       .option("query","MATCH (h:Home_Loan)  RETURN\
               h.home_loan_id as home_loan_id,h.tenure as tenure,h.orig_dt as orig_dt,h.maturity_dt as maturity_dt,h.loan_status as loan_status")\
        .load()

display(df_read_home_loan_data)
       

home_loan_id,tenure,orig_dt,maturity_dt,loan_status
HL868849,25,29-07-2008,28-06-2033,InForce
HL537584,25,27-03-2015,24-02-2040,InForce
HL464229,12,25-05-2016,05-10-2028,InForce
HL527310,25,06-08-2015,07-05-2040,InForce
HL159536,10,12-11-2017,31-10-2027,InForce
HL898643,25,16-09-2010,16-08-2035,InForce
HL862570,12,10-05-2011,25-04-2023,InForce
HL208697,25,12-07-2010,06-11-2035,InForce
HL681200,20,07-11-2011,13-10-2031,InForce
HL805092,15,07-11-2009,19-10-2024,InForce


###Installing graphframes library

In [0]:
!pip install graphframes

     |████████████████████████████████| 154 kB 6.9 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
